# Exercício 2 (BIKE) - Trabalho Prático 3

**Grupo 6:** 


Ruben Silva - pg57900

Luís Costa - pg55970

### imports

In [ ]:
import hashlib
import os
from sage.all import GF, PolynomialRing, ceil
import math


### Paramêtros

**Tamanho Polinomial**  $\rightarrow$ $r = 12323$: .

**Row weight** ($w/2 = 71$)  $\rightarrow$ $w = 142$: .

**Error weight** $\rightarrow$ $t = 134$: .

**Length of $m$, $\sigma$, and $K$**. $\rightarrow$ $\ell = 256$: 


In [ ]:
# Parametros
# Tamanho do anel
r = 12323
w = 142    # Row weight
t = 134    # Error weight
ell = 256  # Shared secret size in bits

# Anel R = F₂[X] / (Xʳ - 1)
F2 = GF(2)          # Campo finito com 2 elementos
P = PolynomialRing(F2, 'x')  # Anel de polinômios F₂[x]
x = P.gen()         # Gerador do anel de polinômios
R = P.quotient(x**r - 1)  # Anel quociente F₂[x]/(x^r - 1)

## Funções Auxiliares

`poly_to_bytes`
1. Converte um polinômio em uma sequência de bytes:
    * Garante que os coeficientes tenham o comprimento correto;
    * Transforma em uma string de bits e converte em bytes.

In [ ]:
def poly_to_bytes(p, length):
    # Converter o Coeficiente do polinômio para uma string de bits
    coeffs = p.list() + [0] * (length - len(p.list())) 

    # Juntar os coeficientes em uma string de bits
    bitstr = ''.join(str(int(c)) for c in coeffs)

    # Calcular o comprimento necessário em bytes
    byte_length = ceil(length / 8)

    # Preencher com zeros se necessário
    padded_bitstr = bitstr.ljust(byte_length * 8, '0')

    # Converter a string de bits em bytes
    return bytes(int(padded_bitstr[i:i+8], 2) for i in range(0, len(padded_bitstr), 8))



`xor_bytes`
1. Realiza o XOR entre duas sequências de bytes.

In [ ]:
def xor_bytes(a, b):
    return bytes(x ^ y for x, y in zip(a, b))

`WSHAKE256_PRF`
1. Gera posições distintas para polinômios esparsos usando SHAKE256:

    * Utiliza hashlib.shake_256 para gerar bytes;
    * Converte em inteiros de 32 bits;
    * Calcula posições distintas com base na especificação.

In [ ]:
def WSHAKE256_PRF(seed, length, wt):

    # Shake256
    shake = hashlib.shake_256()
    shake.update(seed)
    stream_bytes = shake.digest(wt * 4)  # wt 32-bit ints

    # Converter o stream de bytes em inteiros
    s = [int.from_bytes(stream_bytes[i*4:(i+1)*4], 'big') for i in range(wt)]
    wlist = []
    present = set()

    # Gerar a lista de posições
    for i in range(wt - 1, -1, -1):
        pos = i + (length - i) * s[i] // 2**32
        if pos not in present and pos < length:
            wlist.append(pos)
            present.add(pos)
        else:
            wlist.append(i)
            present.add(i)
    return wlist

`sample_polynomial_from_positions`
1. Cria um polinômio em $\mathcal{R}$ com coeficientes 1 nas posições indicadas

In [ ]:
def sample_polynomial_from_positions(positions, length):
    # Criar um polinômio com coeficientes 0  de comprimento 'length'
    coeffs = [0] * length

    # Definir os coeficientes 1 nas posições especificadas
    for pos in positions:
        coeffs[pos] = 1
    
    # Criar o polinômio a partir dos coeficientes
    return R(P(coeffs))

### Hashing Functions

`H(m, r, t)`

1. Mapeia uma mensagem de 256 bits para dois polinômios $(\mathbf{e}_0, \mathbf{e}_1)$ com peso total $t$:

    * Usa WSHAKE256_PRF para gerar $t$ posições distintas;
    * Distribui as posições entre $\mathbf{e}_0$ e $\mathbf{e}_1$.

In [ ]:
# Hash function H: {0,1}²⁵⁶ → E_t
def H(m, r, t):
    positions = WSHAKE256_PRF(m, 2 * r, t)
    e0_coeffs = [0] * r
    e1_coeffs = [0] * r
    for pos in positions:
        if pos < r:
            e0_coeffs[pos] = 1
        else:
            e1_coeffs[pos - r] = 1
    e0 = R(P(e0_coeffs))
    e1 = R(P(e1_coeffs))
    return e0, e1

`L(e0, e1, r)`

1. Mapeia dois polinômios para uma string de 256 bits:

    * Converte os polinômios em bytes;
    * Aplica SHA384 e retorna os primeiros 32 bytes.

In [ ]:
def L(e0, e1, r):
    input_bytes = poly_to_bytes(e0, r) + poly_to_bytes(e1, r)
    return hashlib.sha384(input_bytes).digest()[:32]  # 256 bits = 32 bytes


`K(m, c, r)`

1. Mapeia a mensagem e o ciphertext para a chave partilhada:

    * Concatena $m$, $c_0$ (em bytes) e $c_1$;
    * Usa SHA384 para gerar 256 bits.

In [ ]:
def K(m, c, r):
    c0, c1 = c
    input_bytes = m + poly_to_bytes(c0, r) + c1
    return hashlib.sha384(input_bytes).digest()[:32]

## BGF (Black-Gray-Flip)

`getHammingWeight`
1. Calcula o peso de Hamming (número de bits 1) de uma sequência binária de comprimento especificado.

In [ ]:
def getHammingWeight(tmp, length):
    return sum(int(tmp[i]) for i in range(length))

`recompute_syndrome`
1. Atualiza o síndrome $  s  $ após a inversão de um bit na posição pos do vetor de erro, considerando as matrizes de paridade compactadas $  \mathbf{h}_0  $ e $  \mathbf{h}_1  $.

In [ ]:
def recompute_syndrome(s, pos, h0_compact, h1_compact, r):

    # se pos < r, atualiza o síndrome usando h0_compact
    if pos < r:
        for j in range(len(h0_compact)):
            if h0_compact[j] <= pos:
                s[(pos - h0_compact[j]) % r] ^= 1
            else:
                s[(r - h0_compact[j] + pos) % r] ^= 1

    # se pos >= r, atualiza o síndrome usando h1_compact
    else:
        pos -= r
        for j in range(len(h1_compact)):
            if h1_compact[j] <= pos:
                s[(pos - h1_compact[j]) % r] ^= 1
            else:
                s[(r - h1_compact[j] + pos) % r] ^= 1

`ctr`
1. Calcula o contador de paridade insatisfeita (número de verificações de paridade violadas) para uma coluna específica da matriz de paridade, dado o síndrome.

In [ ]:
def ctr(h_compact_col, position, s, r):
    count = 0
    for i in range(len(h_compact_col)):
        index = (h_compact_col[i] + position) % r
        if s[index]:
            count += 1
    return count

`getCol`
1. Converte uma representação compactada de uma linha da matriz de paridade ($\mathbf{h}_0$ ou $\mathbf{h}_1$) em uma representação compactada da coluna correspondente.

In [ ]:
def getCol(h_compact_row, r):
    h_compact_col = [0] * len(h_compact_row)
    if h_compact_row[0] == 0:
        h_compact_col[0] = 0
        for i in range(1, len(h_compact_row)):
            h_compact_col[i] = r - h_compact_row[len(h_compact_row) - i]
    else:
        for i in range(len(h_compact_row)):
            h_compact_col[i] = r - h_compact_row[len(h_compact_row) - 1 - i]
    return h_compact_col

`flipAdjustedErrorPosition`
1. Inverte um bit na posição ajustada do vetor de erro, considerando a estrutura do BIKE.

In [ ]:
def flipAdjustedErrorPosition(e, position, r):
    adjustedPosition = position
    if position != 0 and position != r:
        adjustedPosition = (position > r) and ((2 * r - position) + r) or (r - position)
    e[adjustedPosition] ^= 1

`BFMaskedIter`
1. Executa uma iteração mascarada do algoritmo de decodificação, invertendo bits no vetor de erro com base em uma máscara e um limiar.

In [ ]:
def BFMaskedIter(e, s, mask, T, h0_compact, h1_compact, h0_compact_col, h1_compact_col, r):
    pos = [0] * (2 * r)
    for j in range(r):
        counter = ctr(h0_compact_col, j, s, r)
        if counter >= T and mask[j]:
            flipAdjustedErrorPosition(e, j, r)
            pos[j] = 1
    for j in range(r, 2 * r):
        counter = ctr(h1_compact_col, j - r, s, r)
        if counter >= T and mask[j]:
            flipAdjustedErrorPosition(e, j, r)
            pos[j] = 1
    for j in range(2 * r):
        if pos[j]:
            recompute_syndrome(s, j, h0_compact, h1_compact, r)


`BFIter`
1. Executa uma iteração principal do algoritmo BGF, classificando posições como "black" ou "gray" com base em contadores de paridade e ajustando o vetor de erro.

In [ ]:
def BFIter(e, black, gray, s, T, tau, h0_compact, h1_compact, h0_compact_col, h1_compact_col, r):
    pos = [0] * (2 * r)
    for j in range(r):
        counter = ctr(h0_compact_col, j, s, r)
        if counter >= T:
            flipAdjustedErrorPosition(e, j, r)
            pos[j] = 1
            black[j] = 1
        elif counter >= T - tau:
            gray[j] = 1
    for j in range(r, 2 * r):
        counter = ctr(h1_compact_col, j - r, s, r)
        if counter >= T:
            flipAdjustedErrorPosition(e, j, r)
            pos[j] = 1
            black[j] = 1
        elif counter >= T - tau:
            gray[j] = 1
    for j in range(2 * r):
        if pos[j]:
            recompute_syndrome(s, j, h0_compact, h1_compact, r)


`BGF_decoder`
1. Implementa o decodificador BGF completo, executando iterações para corrigir erros no vetor de erro com base no síndrome e nas matrizes de paridade.

In [ ]:
def BGF_decoder(e, s, h0_compact, h1_compact, r, w, t, NbIter=5, tau=3):
    e = [0] * (2 * r)
    h0_compact_col = getCol(h0_compact, r)
    h1_compact_col = getCol(h1_compact, r)
    black = [0] * (2 * r)
    gray = [0] * (2 * r)
    for i in range(1, NbIter + 1):
        black = [0] * (2 * r)
        gray = [0] * (2 * r)
        S = getHammingWeight(s, r)
        T = max(math.ceil(0.0069722 * S + 13.530), 36)  # Threshold ajustado para nível 1
        BFIter(e, black, gray, s, T, tau, h0_compact, h1_compact, h0_compact_col, h1_compact_col, r)
        if i == 1:
            T_mask = (w // 2 + 1) // 2 + 1
            BFMaskedIter(e, s, black, T_mask, h0_compact, h1_compact, h0_compact_col, h1_compact_col, r)
            BFMaskedIter(e, s, gray, T_mask, h0_compact, h1_compact, h0_compact_col, h1_compact_col, r)
        if getHammingWeight(s, r) == 0:
            print("Decodificação bem-sucedida!\n")
            return e
            
    return None  # Falha na decodificação

# BIKE Implementatioon

A função `KeyGen`  é responsável por gerar o par de chaves pública e privada:

1. Gerar Componentes da Chave Privada:
    * Criar dois polinómios esparsos, $\mathbf{h}_0$ e $\mathbf{h}_1$, no anel $\mathcal{R} $;
    * Cada polinómio deve ter exatamente $w/2$ coeficientes iguais a 1, onde $w$ é o peso da linha;
    * Utilizar uma função pseudoaleatória, como `WSHAKE256-PRF`, com uma semente, para escolher $w/2$ posições distintas para os coeficientes 1 em cada polinómio.


2. Calcular a Chave Pública:

    * Calcular $\mathbf{h} = \mathbf{h}_1 \cdot \mathbf{h}_0^{-1}$ em $\mathcal{R}$.
    * Para que $\mathbf{h}_0$ seja invertível, $w/2$ é definido como um número ímpar, garantindo esta propriedade no anel.


3. Gerar uma String Aleatória:

    * Produzir uma string $\sigma \in \{0,1\}^{256}$ de 256 bits, utilizando um gerador de números aleatórios seguro. Esta string é usada como recurso de reserva no desencapsulamento, caso a decodificação falhe.

In [ ]:
# Key Generation
def KeyGen(r, w):
    seed_h0 = os.urandom(32)
    positions_h0 = WSHAKE256_PRF(seed_h0, r, w // 2)
    h0 = sample_polynomial_from_positions(positions_h0, r)
    
    seed_h1 = os.urandom(32)
    positions_h1 = WSHAKE256_PRF(seed_h1, r, w // 2)
    h1 = sample_polynomial_from_positions(positions_h1, r)
    
    h = h1 * h0.inverse_of_unit()  # Compute h = h₁ * h₀⁻¹ in R
    sigma = os.urandom(32)
    
    sk = (h0, h1, sigma)  # Private key
    pk = h               # Public key
    return sk, pk

A função `Encaps`  é responsável por gerar uma chave secreta partilhada e o seu ciphertext utilizando a chave pública.

1. Escolher uma Mensagem Aleatória:
    * Gerar uma string aleatória $m \in \{0,1\}^{256}$ de 256 bits com um gerador seguro.


2. Gerar Vetores de Erro:

    * Aplicar uma função hash $\mathbf{H}$ (ex.: SHAKE256) a $m$ para criar os vetores de erro $(\mathbf{e}_0, \mathbf{e}_1)$, com peso total de Hamming $t$ (ex.: $t = 134$ para nível 1).


3. Calcular o Ciphertext:

    * $c_0 = \mathbf{e}_0 + \mathbf{e}_1 \cdot \mathbf{h}$ em $\mathcal{R}$;
    * $c_1 = m \oplus \mathbf{L}(\mathbf{e}_0, \mathbf{e}_1)$, onde $\mathbf{L}$ (ex.: SHA384) mapeia os vetores de erro para uma string de 256 bits;
    * O ciphertext é $c = (c_0, c_1)$.

4. Calcular a Chave Secreta:

    * $K = \mathbf{K}(m, c)$, onde $\mathbf{K}$ (ex.: SHA384) faz hash de $m$ e $c$.

In [ ]:
# Encapsulation
def Encaps(pk, r, t):
    h = pk
    m = os.urandom(32)  # m ←$ {0,1}²⁵⁶
    e0, e1 = H(m, r, t)
    c0 = e0 + e1 * h
    c1 = xor_bytes(m, L(e0, e1, r))
    c = (c0, c1)
    K_shared = K(m, c, r)
    return K_shared, c

A função `Decaps`  é responsável por recuperar a chave secreta partilhada a partir do ciphertext usando a chave privada.

1. Decodificar o Ciphertext:
    * Calcular o síndrome $s = c_0 \cdot \mathbf{h}_0$ em $\mathcal{R}$.
    * Usar um decodificador (BGF) para recuperar os vetores de erro $\mathbf{e}' = (\mathbf{e}_0', \mathbf{e}_1')$.

2. Recuperar a Mensagem:

    * $m' = c_1 \oplus \mathbf{L}(\mathbf{e}_0', \mathbf{e}_1')$, com $\mathbf{L}$ igual à usada em `Encaps`


3. Verificar e Calcular a Chave:

    * Verificar se $\mathbf{e}' = \mathbf{H}(m')$ (usando a mesma $\mathbf{H}$ de `Encaps`)
    * Se verdadeiro: $K = \mathbf{K}(m', c)$.
    * Se falso (falha na decodificação): $K = \mathbf{K}(\sigma, c)$, usando $\sigma$ como reserva.

In [ ]:
# Decapsulation (with simulated decoder)
def Decaps(sk, c, r, t):
    h0, h1, sigma = sk
    c0, c1 = c
    
    # Simulate decoder: assume it correctly recovers e0 and e1
    h0 = sk[0]  # Primeira parte da chave privada
    c0 = c[0]   # Primeira parte do ciphertext
    s = (c0 * h0).list()  # Síndrome como lista
    s = [int(coeff) for coeff in s]  # Converter para inteiros Python
    h0_list = [int(coeff) for coeff in sk[0].list()]
    h1_list = [int(coeff) for coeff in sk[1].list()]

    dec = BGF_decoder([0] * (2 * r), r * [0], h0_list, h1_list, r, w, t)
    e0_prime = R(P(dec[:r]))
    e1_prime = R(P(dec[r:2 * r]))
    
    m_prime = xor_bytes(c1, L(e0_prime, e1_prime, r))
    
    e0_check, e1_check = H(m_prime, r, t)
    c0_prime = e0_check + e1_check * (h1 * h0.inverse_of_unit())
    c1_prime = xor_bytes(m_prime, L(e0_check, e1_check, r))
    
    if c0 == c0_prime and c1 == c1_prime:
        return K(m_prime, c, r)
    else:
        return K(sigma, c, r)

### Run

In [ ]:
print("======= BIKE - Key Encapsulation Mechanism ======= \n")
sk, pk = KeyGen(r, w)
print("Chave Gerada com sucesso.")
print("Chave Pública:", pk)
print("Chave Privada:", sk)

# Encapsulate
K_enc, c = Encaps(pk, r, t)
print("\nEncapsulação completa!")

# Decapsulate
print("\n======= DESCOMPACTANDO =======")
K_dec = Decaps(sk, c, r, t)

======= BIKE - Key Encapsulation Mechanism ======= 

Chave Gerada com sucesso.
Chave Pública: xbar^12322 + xbar^12318 + xbar^12315 + xbar^12314 + xbar^12313 + xbar^12311 + xbar^12309 + xbar^12308 + xbar^12306 + xbar^12300 + xbar^12299 + xbar^12298 + xbar^12295 + xbar^12294 + xbar^12293 + xbar^12292 + xbar^12290 + xbar^12287 + xbar^12285 + xbar^12284 + xbar^12278 + xbar^12276 + xbar^12275 + xbar^12274 + xbar^12273 + xbar^12268 + xbar^12266 + xbar^12264 + xbar^12263 + xbar^12262 + xbar^12260 + xbar^12258 + xbar^12253 + xbar^12252 + xbar^12251 + xbar^12249 + xbar^12248 + xbar^12247 + xbar^12246 + xbar^12241 + xbar^12240 + xbar^12239 + xbar^12230 + xbar^12229 + xbar^12228 + xbar^12225 + xbar^12223 + xbar^12220 + xbar^12213 + xbar^12212 + xbar^12210 + xbar^12208 + xbar^12207 + xbar^12206 + xbar^12204 + xbar^12203 + xbar^12202 + xbar^12200 + xbar^12199 + xbar^12198 + xbar^12196 + xbar^12194 + xbar^12191 + xbar^12190 + xbar^12185 + xbar^12184 + xbar^12183 + xbar^12182 + xbar^12181 + xbar^1218


Encapsulação completa!

======= DESCOMPACTANDO =======
Decodificação bem-sucedida!

